In [1]:
!hdfs dfs -rm -r -skipTrash hdfs:///user/denton/reddit_snappy.parquet
sqlContext.setConf("spark.sql.parquet.compression.codec", "snappy")

rm: `hdfs:///user/denton/reddit_snappy.parquet': No such file or directory


In [1]:
!hdfs dfs -ls -R -h hdfs:///datasets/reddit_data

drwxr-xr-x   - denton hdfs          0 2015-07-29 14:30 hdfs:///datasets/reddit_data/2007
-rw-r--r--   2 denton hdfs     12.7 M 2015-07-29 14:30 hdfs:///datasets/reddit_data/2007/RC_2007-10.bz2
-rw-r--r--   2 denton hdfs     27.4 M 2015-07-29 14:30 hdfs:///datasets/reddit_data/2007/RC_2007-11.bz2
-rw-r--r--   2 denton hdfs     29.7 M 2015-07-29 14:30 hdfs:///datasets/reddit_data/2007/RC_2007-12.bz2
drwxr-xr-x   - denton hdfs          0 2015-07-29 14:31 hdfs:///datasets/reddit_data/2008
-rw-r--r--   2 denton hdfs     36.2 M 2015-07-29 14:30 hdfs:///datasets/reddit_data/2008/RC_2008-01.bz2
-rw-r--r--   2 denton hdfs     35.4 M 2015-07-29 14:31 hdfs:///datasets/reddit_data/2008/RC_2008-02.bz2
-rw-r--r--   2 denton hdfs     37.1 M 2015-07-29 14:31 hdfs:///datasets/reddit_data/2008/RC_2008-03.bz2
-rw-r--r--   2 denton hdfs     38.0 M 2015-07-29 14:31 hdfs:///datasets/reddit_data/2008/RC_2008-04.bz2
-rw-r--r--   2 denton hdfs     43.2 M 2015-07-29 14:31 hdfs:///datasets/reddit_data/2008/RC_20

In [10]:
!hdfs dfs -du -h -s hdfs:///datasets/reddit_data

149.6 G  hdfs:///datasets/reddit_data


In [3]:
%%time
reddit = sqlContext.read.json('hdfs:///datasets/reddit_data/*/*.bz2') # not using cache() function

CPU times: user 44.1 ms, sys: 14 ms, total: 58.1 ms
Wall time: 3min 55s


In [4]:
reddit.printSchema()

root
 |-- approved_by: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- banned_by: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- mod_reports: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- num_reports: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- r

This bit me: http://stackoverflow.com/questions/33067519/spark-sql-no-input-paths-specified-in-job-but-can-printschema
Might also be relevant: http://stackoverflow.com/questions/28817940/recursively-fetch-file-contents-from-subdirectories-using-sc-textfile

In [5]:
%%time
reddit.count()

CPU times: user 24.2 ms, sys: 8.29 ms, total: 32.5 ms
Wall time: 2min 48s


1659361605

In [5]:
%%time
print("reddit partitions = ", reddit.rdd.getNumPartitions())

('reddit partitions = ', 1239)
CPU times: user 303 µs, sys: 3.1 ms, total: 3.4 ms
Wall time: 420 ms


We are using 240 executors with 7 cores each for a total of 1680 cores. Let's repartition to 1680 to get more parallelism.

In [6]:
%%time
reddit = reddit.repartition(1680)

CPU times: user 1.23 ms, sys: 1.89 ms, total: 3.11 ms
Wall time: 23.5 ms


In [7]:
%%time
print("reddit partitions = ", reddit.rdd.getNumPartitions())

('reddit partitions = ', 1680)
CPU times: user 2.52 ms, sys: 990 µs, total: 3.51 ms
Wall time: 289 ms


In [ ]:
%%time
reddit.count()

In [8]:
%%time
reddit.write.save("hdfs:///user/denton/reddit_snappy.parquet", format="parquet")

CPU times: user 125 ms, sys: 39.6 ms, total: 165 ms
Wall time: 11min 53s


In [9]:
!hdfs dfs -du -h -s hdfs:///user/denton/reddit_snappy.parquet

269.4 G  hdfs:///user/denton/reddit_snappy.parquet


!hdfs dfs -rm -r -skipTrash hdfs:///user/denton/reddit_lzo.parquet
sqlContext.setConf("spark.sql.parquet.compression.codec", "lzo")

%%time
reddit.write.save("hdfs:///user/denton/reddit_lzo.parquet", format="parquet")

In [1]:
%%time
reddit = sqlContext.read.load("hdfs:///user/denton/reddit_snappy.parquet").cache()

CPU times: user 8.12 ms, sys: 3.32 ms, total: 11.4 ms
Wall time: 8.47 s


In [2]:
%%time
reddit.count()

CPU times: user 34.7 ms, sys: 12.9 ms, total: 47.6 ms
Wall time: 2min 45s


1659361605

In [3]:
%%time
reddit.count()

CPU times: user 2.27 ms, sys: 1.26 ms, total: 3.53 ms
Wall time: 3.07 s


1659361605

In [5]:
%%time
reddit.groupBy("subreddit").count().sort("count", ascending=False).show(1000)

+--------------------+---------+
|           subreddit|    count|
+--------------------+---------+
|           AskReddit|184540520|
|               funny| 49967219|
|                pics| 47208205|
|              gaming| 32202209|
|                 WTF| 29779435|
|     leagueoflegends| 29706965|
|       AdviceAnimals| 27339965|
|            politics| 22904996|
|              videos| 21235667|
|           worldnews| 19687581|
|       todayilearned| 18508173|
|                IAmA| 18298109|
|                 nfl| 17116723|
|             atheism| 16138248|
|               trees| 14312315|
|                 nba| 12929889|
|          reddit.com| 12560130|
|              soccer| 11566836|
|              movies| 10617752|
|               DotA2| 10179173|
|                news|  9881729|
|            gonewild|  9383775|
|              hockey|  9336295|
|          technology|  8743613|
| fffffffuuuuuuuuuuuu|  8426373|
|                 CFB|  7569656|
|                 aww|  7259234|
|       Sq

In [9]:
%%time
reddit.registerTempTable("reddit")

CPU times: user 498 µs, sys: 104 µs, total: 602 µs
Wall time: 908 µs


In [11]:
%%time
cats_df = sqlContext.sql("SELECT * FROM reddit WHERE body LIKE '% cat %' OR body LIKE '% cats %'")

CPU times: user 2.56 ms, sys: 319 µs, total: 2.87 ms
Wall time: 48.9 ms


In [12]:
cats_df.count()

3324590

In [14]:
cats_df.show(10)

+-----------+--------+------------------+----------------------+-----------------+---------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+-----+---------+-----------+----------+-----------+----------+--------------+-------+--------------+------------+-----+-----+------------+-----------------+------------+---+------------+
|approved_by|archived|            author|author_flair_css_class|author_flair_text|banned_by|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|likes|  link_id|mod_reports|      name|num_reports| parent_id|removal_reason|replies|report_reasons|retrieved_on|saved|score|score_hidden|        subreddit|subreddit_id|ups|user_reports|
+-----------+--------+------------------+----------------------+-----------------+---------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+-----+---------+----

In [29]:
%%time
bacon_df = sqlContext.sql("SELECT * FROM reddit WHERE body LIKE '% bacon %'")

CPU times: user 506 µs, sys: 2.01 ms, total: 2.52 ms
Wall time: 24.5 ms


In [30]:
%%time
bacon_df.count()

CPU times: user 3.14 ms, sys: 1.29 ms, total: 4.43 ms
Wall time: 6.76 s


544006

In [33]:
%%time
clemson_df = sqlContext.sql("SELECT * FROM reddit WHERE subreddit LIKE '%Clemson%'")

CPU times: user 502 µs, sys: 2.08 ms, total: 2.58 ms
Wall time: 21.7 ms


In [34]:
%%time
clemson_df.count()

CPU times: user 1.35 ms, sys: 1.35 ms, total: 2.7 ms
Wall time: 2.18 s


22809

In [36]:
clemson_df.groupBy('author').count().sort('count', ascending=False).show(1000)

+--------------------+-----+
|              author|count|
+--------------------+-----+
|           [deleted]| 2548|
|             Hotwir3|  424|
|             MRatata|  414|
|           swimmer33|  212|
|             jdubrow|  195|
|         ArtimusPyle|  190|
|            wcrisler|  181|
|               pmstc|  181|
|         silentbobsc|  180|
|          keineliebe|  145|
|      trainmaster611|  141|
|       clemtiger2011|  138|
|      you_sick_ducks|  129|
|              ycpa68|  128|
|             Milagre|  128|
|              tupper|  124|
|            veringer|  115|
|       fuckyeahashmo|  114|
|         WinterCharm|  110|
|          PilotJ0nes|  108|
|          ReconTiger|  105|
|         flirtybirdy|  104|
|             KeeganB|  103|
|        vinylasphalt|  103|
|          dirtmcgurk|  100|
|             bschroe|   97|
| burn_all_the_things|   95|
|           someones1|   93|
|              Flucks|   90|
|          rocknrun18|   87|
|           TheBlayer|   87|
|      imsoupe